In [1]:
import data_cnn as data
from tensorflow.python.framework import ops
ops.reset_default_graph()
import numpy as np
import tensorflow as tf

In [2]:
dictLabels = data.getAllLabels('/media/data/gomez/Fully_con_NN_Data_input/split1Labels.mat')
dictData = data.getAllData('/media/data/gomez/Fully_con_NN_Data_input/Input_data1.mat')

dataTrain = dictData['train']
dataTest = dictData['test']
labelsTrain = dictLabels['train']
labelsTest = dictLabels['test']

Getting a dictionary with the labels of train and test
Retrieving labels from the file /media/data/gomez/Fully_con_NN_Data_input/split1Labels.mat from train data
Retrieving labels from the file /media/data/gomez/Fully_con_NN_Data_input/split1Labels.mat from test data
Getting a dictionary with the data of train and test
Retrieving data from the file /media/data/gomez/Fully_con_NN_Data_input/Input_data1.mat from train data
Retrieving data from the file /media/data/gomez/Fully_con_NN_Data_input/Input_data1.mat from test data
calculating max frame sequence in train and test data
Maximum number of rows found and it is: 28
from train data
from test data


In [3]:
#Parameters of training and viewing probabilities of the 
training_epochs = 10
drop_out_prob = 0.5
display_step = 10
iterations = 9000
learning_rate = 0.00001
tb_logs_path = '/home/gomez/tb_net_result'

In [4]:
batch_size,shpeData = dataTrain.shape
_,shpeLabl = labelsTrain.shape
print('training', shpeData,shpeLabl,batch_size,_)
'''batch_size,shpeData = dataTest.shape
_,shpeLabl = labelsTest.shape
print('test',shpeData,shpeLabl,batch_size,_)'''

#Modify the data according to the given shape above
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,840])
    y_ = tf.placeholder(tf.float32,[None,12])    

training 840 12 227 227


In [5]:
#Weight initialization

def weight_variable(shape, name, var_type): #Change the type in order to specify the variable to use    
    with tf.name_scope('weight_' + name):
        initial = tf.random_uniform(shape)
        if(var_type == 'normal'):
            initial = tf.random_normal(shape, stddev=0.1)
        elif(var_type == 'trunc'): #As seen in the beginning
            initial = tf.truncated_normal(shape, stddev=0.1)            
        w_variable = tf.Variable(initial)
        tf.scalar_summary('_weight_' + name, w_variable)
        return w_variable

def bias_variable(shape, name):
    with tf.name_scope('bias_' + name):
        initial = tf.constant(0.1, shape=shape)
        b_variable = tf.Variable(initial)
        tf.scalar_summary('_bias_' + name,b_variable)
        return b_variable

#Convolution and pooling (Martin: stride 30 for convolution, article: max_pooling of 7 with stride 7)

def conv1d(x,w):
    with tf.name_scope('Conv1d'):
        convolution = tf.nn.conv1d(x,w,stride=30, padding='SAME')    
        tf.scalar_summary('_Conv1d_',convolution)
        return convolution

def max_pool_7(x):
    with tf.name_scope('max_pool_stride7'):
        max_pool7 = tf.nn.max_pool(x,ksize=[1,1,7,1],strides=[1,1,7,1], padding='SAME') #There is just max_pool in 2d and 3d so reshape and reshape
        tf.scalar_summary('_max_pool_stride7_',max_pool7)
        return max_pool7

In [6]:
#First Layer

w_conv1 = weight_variable([30,1,32], 'conv1', 'normal') #Kernel size of 30 and from 1 input to 32 neurons
b_conv1 = bias_variable([32], 'conv1') #32 biases going to different neurons

x_data = tf.reshape(x, [-1,shpeData,1]) #Image converted in 3d

h_conv1 = tf.nn.relu(conv1d(x_data, w_conv1) + b_conv1)

print(h_conv1)

'''data: [3d] -> maxpooling(data): [4d] -> reconversion: [3d]'''

# h_conv1_4d:[batch_size,1 (1d data), 28 (840/30 = after convolution 30 (filter and stride) ), 32 (the output to the new network)]

h_conv1_4d = tf.reshape(h_conv1,[-1,1,int(shpeData/30),1]) #Error is here --> Sure 32 is here???

h_pool1 = max_pool_7(h_conv1_4d)

print(h_pool1)

#2 rshp_h_pool1:[batch_size, 4 (28/7 -> 28 from previous convolution/ 7 from maxpool filter and stride), 32 (output network)]

rshp_h_pool1 = tf.reshape(h_pool1,[-1,int((shpeData/30)/7),1]) #Error is here --> Sure 32 is here???
print(rshp_h_pool1)

Tensor("Relu:0", shape=(?, 28, 32), dtype=float32)
Tensor("max_pool_stride7/MaxPool:0", shape=(?, 1, 4, 1), dtype=float32)
Tensor("Reshape_2:0", shape=(?, 4, 1), dtype=float32)


In [7]:
#Fully Connected layer 1

W_fc1 = weight_variable([4 * 32,128], 'fc1','uniform') #weights converted from the dimensions
b_fc1 = bias_variable([128], 'fc1')

h_pool2_flat = tf.reshape(rshp_h_pool1, [-1, 4*32])
print(h_pool2_flat)
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
print(h_fc1)

Tensor("Reshape_3:0", shape=(?, 128), dtype=float32)
Tensor("Relu_1:0", shape=(?, 128), dtype=float32)


In [8]:
#Fully Connected layer 2

W_fc2 = weight_variable([128,256], 'fc2', 'uniform') #weights converted to 
b_fc2 = bias_variable([256], 'fc2') #Bias values going to the neurons

h_fc2 = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2) #Relu #2 -> Check if this was right -> Should be (before the h_pool2 was)
print(h_fc2)

Tensor("Relu_2:0", shape=(?, 256), dtype=float32)


In [9]:
# Dropout trick 

keep_prob = tf.placeholder("float")
h_fc2_drop = tf.nn.dropout(h_fc2,keep_prob)
print(h_fc2_drop)

# Output layer

W_fc3 = weight_variable([256,12], 'fc3', 'uniform')
b_fc3 = bias_variable([12], 'fc3')

eps = tf.constant(0.00001, shape=[12]) #Constant value added to prevent underflow (probability of having zero terms)
y_conv = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3) + eps
print(y_conv)

Tensor("dropout/mul_1:0", shape=(?, 256), dtype=float32)
Tensor("add_4:0", shape=(?, 12), dtype=float32)


In [10]:
#Train and evaluate
with tf.name_scope('cross_entropy'):
    cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.arg_max(y_conv,1),tf.arg_max(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

tf.scalar_summary('cost', cross_entropy)
tf.scalar_summary('accuracy', accuracy)

<tf.Tensor 'ScalarSummary_1:0' shape=() dtype=string>

In [11]:
#Merging all summaries
summary_op = tf.merge_all_summaries()

In [12]:
#Starting a new session
'''init = tf.initialize_all_variables()
sess = tf.InteractiveSession()
sess.run(init)
writer = tf.train.SummaryWriter(tb_logs_path, graph=tf.get_default_graph())'''

'init = tf.initialize_all_variables()\nsess = tf.InteractiveSession()\nsess.run(init)\nwriter = tf.train.SummaryWriter(tb_logs_path, graph=tf.get_default_graph())'

In [13]:
'''for i in range(iterations):    
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:dataTrain, y_:labelsTrain, keep_prob: 0.5})
        #summary = summary_op.eval(feed_dict={x:dataTrain, y_:labelsTrain, keep_prob: 0.5})
        print("step %d, training accuracy %g"%(i,train_accuracy))
        #writer.add_summary(summary, epoch)
    train_step.run(feed_dict={x:dataTrain, y_:labelsTrain, keep_prob: 0.5})
    #writer.add_summary
print("test accuracy %g"%accuracy.eval(feed_dict={x:dataTest, y_:labelsTest, keep_prob: 0.5}))'''

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step < iterations:
        sess.run(train_step, feed_dict={x:dataTrain, y_:labelsTrain, keep_prob: drop_out_prob})
        if step % display_step == 0:
            #Calculates batch accuracy
            train_acc, loss = sess.run([accuracy, cost], feed_dict={x:dataTrain, y_:labelsTrain, keep_prob: drop_out_prob})
            print(' Iteration: ' + str(step*batch_size) + ' , loss: ' + '{:.6f}'.format(loss) + ' Accuracy: ' + '{:.6f}'.format(train_acc))
        step +=1
    print('Optimization Finished')
    
    test_acc = sess.run(accuracy, feed_dict={x:dataTest, y_:labelsTest, keep_prob:drop_out_prob})
    print('And test accuracy is:',test_acc)

 Iteration: 2270 , loss: 2.513002 Accuracy: 0.105727
 Iteration: 4540 , loss: 2.499786 Accuracy: 0.118943
 Iteration: 6810 , loss: 2.521813 Accuracy: 0.096916
 Iteration: 9080 , loss: 2.526218 Accuracy: 0.092511
 Iteration: 11350 , loss: 2.539434 Accuracy: 0.079295
 Iteration: 13620 , loss: 2.543833 Accuracy: 0.074890
 Iteration: 15890 , loss: 2.508597 Accuracy: 0.110132
 Iteration: 18160 , loss: 2.504192 Accuracy: 0.114537
 Iteration: 20430 , loss: 2.543839 Accuracy: 0.074890
 Iteration: 22700 , loss: 2.517407 Accuracy: 0.101322
 Iteration: 24970 , loss: 2.517407 Accuracy: 0.101322
 Iteration: 27240 , loss: 2.570271 Accuracy: 0.048458
 Iteration: 29510 , loss: 2.565866 Accuracy: 0.052863
 Iteration: 31780 , loss: 2.508597 Accuracy: 0.110132
 Iteration: 34050 , loss: 2.548244 Accuracy: 0.070485
 Iteration: 36320 , loss: 2.552650 Accuracy: 0.066079
 Iteration: 38590 , loss: 2.517407 Accuracy: 0.101322
 Iteration: 40860 , loss: 2.526218 Accuracy: 0.092511
 Iteration: 43130 , loss: 2.5218